System information (for reproducibility):

In [2]:
versioninfo()

Julia Version 1.10.3
Commit 0b4590a5507 (2024-04-30 10:59 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, apple-m1)
Threads: 8 default, 0 interactive, 4 GC (on 8 virtual cores)
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [3]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2024spring/slides/19-easylineq`


Status `~/Documents/github.com/ucla-biostat-257/2024spring/slides/19-easylineq/Project.toml`
  [6e4b80f9] BenchmarkTools v1.5.0
  [42fd0dbc] IterativeSolvers v0.9.4
  [b51810bb] MatrixDepot v1.0.11
  [af69fa37] Preconditioners v0.6.1
  [b8865327] UnicodePlots v3.6.4
  [efce3f68] WoodburyMatrices v1.0.0
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [2f01184e] SparseArrays v1.10.0


# Introduction

Consider $\mathbf{A} \mathbf{x} = \mathbf{b}$, $\mathbf{A} \in \mathbb{R}^{n \times n}$. Or, consider matrix inverse (if you want). $\mathbf{A}$ can be huge. Keep massive data in mind: 1000 Genome Project, NetFlix, Google PageRank, finance, spatial statistics, ... We should be alert to many easy linear systems. 

Don't blindly use `A \ b` and `inv` in Julia or `solve` function in R. **Don't waste computing resources by bad choices of algorithms!**

## Diagonal matrix

Diagonal $\mathbf{A}$: $n$ flops. Use `Diagonal` type of Julia.

In [4]:
using BenchmarkTools, LinearAlgebra, Random

# generate random data
Random.seed!(257)
n = 1000
A = diagm(0 => randn(n)) # a diagonal matrix stored as Matrix{Float64}
b = randn(n);

In [5]:
# should give link to the source code
@which A \ b

\(A::AbstractMatrix, B::AbstractVecOrMat)
     @ LinearAlgebra ~/.julia/juliaup/julia-1.10.3+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/LinearAlgebra/src/generic.jl:1110

In [6]:
# check `istril(A)` and `istriu(A)` (O(n^2)), then call `Diagonal(A) \ b` (O(n))
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  363.833 μs …  3.252 ms  ┊ GC (min … max): 0.00% … 87.58%
 Time  (median):     397.334 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   391.586 μs ± 39.063 μs  ┊ GC (mean ± σ):  0.12% ±  1.20%

    ▂                        █                                  
  ▃▄█▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▂▅▅█▅▄▆▂▂▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  364 μs          Histogram: frequency by time          438 μs <

 Memory estimate: 16.00 KiB, allocs estimate: 2.

In [7]:
# O(n) computation, no extra array allocation
@benchmark Diagonal($A) \ $b

BenchmarkTools.Trial: 10000 samples with 28 evaluations.
 Range (min … max):  928.571 ns … 66.769 μs  ┊ GC (min … max):  0.00% … 95.79%
 Time  (median):       1.610 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.048 μs ±  4.533 μs  ┊ GC (mean ± σ):  21.55% ±  9.36%

                      ▄▄▅▇█▃▃                                   
  ▇▃▁▄▁▁▁▁▁▁▁▁▁▁▁▂▂▄▃▅███████▇▆▆▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  929 ns          Histogram: frequency by time         2.61 μs <

 Memory estimate: 16.00 KiB, allocs estimate: 2.

## Bidiagonal, tridiagonal, and banded matrices

Bidiagonal, tridiagonal, or banded $\mathbf{A}$: Band LU, band Cholesky, ... roughly $O(n)$ flops.   
* Use [`Bidiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.Bidiagonal), [`Tridiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.Tridiagonal), [`SymTridiagonal`](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.SymTridiagonal) types of Julia.

In [8]:
Random.seed!(257) 

n  = 1000
dv = randn(n)
ev = randn(n - 1)
b  = randn(n) # rhs
# symmetric tridiagonal matrix
A  = SymTridiagonal(dv, ev)

1000×1000 SymTridiagonal{Float64, Vector{Float64}}:
 0.679063   0.817275    ⋅        …    ⋅          ⋅          ⋅ 
 0.817275   1.24568   -0.527485       ⋅          ⋅          ⋅ 
  ⋅        -0.527485  -1.21007        ⋅          ⋅          ⋅ 
  ⋅          ⋅         0.187263       ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅        …    ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
 ⋮                               ⋱                        
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅          ⋅          ⋅             ⋅          ⋅          ⋅ 
  ⋅    

In [9]:
# convert to a full matrix
Afull = Matrix(A)

# LU decomposition (2/3) n^3 flops!
@benchmark $Afull \ $b

BenchmarkTools.Trial: 1049 samples with 1 evaluation.
 Range (min … max):  3.802 ms … 25.621 ms  ┊ GC (min … max): 0.00% … 19.41%
 Time  (median):     4.228 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.768 ms ±  1.743 ms  ┊ GC (mean ± σ):  3.87% ±  7.44%

    █▅▂ ▁▃▁▁                                                  
  ▆██████████▇▇▆▄▆▄▅▁▅▄▄▅▄▄▆▅▁▄▄▄▁▄▄▅▆▄▄▆▄▁▄▄▁▁▄▄▄▄▄▄▁▁▄▁▅▁▄ ▇
  3.8 ms       Histogram: log(frequency) by time     12.4 ms <

 Memory estimate: 7.65 MiB, allocs estimate: 4.

In [10]:
# specialized algorithm for tridiagonal matrix, O(n) flops
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.125 μs …  1.676 ms  ┊ GC (min … max): 0.00% … 98.56%
 Time  (median):     11.917 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.481 μs ± 25.226 μs  ┊ GC (mean ± σ):  3.44% ±  1.70%

           ██    ▇▅▇▃▁▁  ▃▁▄                                   
  ▆▅▃▁▁▁▃▆▅██▄▃▃▅██████▅▆███▆▅▆▃▅▂▂▂▁▁▁▁▂▂▁▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  10.1 μs         Histogram: frequency by time          16 μs <

 Memory estimate: 24.00 KiB, allocs estimate: 3.

## Triangular matrix

Triangular $\mathbf{A}$: $n^2$ flops to solve linear system.

In [11]:
Random.seed!(257)

n = 1000
A = tril(randn(n, n)) # a lower-triangular matrix stored as Matrix{Float64}
b = randn(n)

# check istril() then triangular solve
@benchmark $A \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  253.541 μs … 570.250 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     258.084 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   261.697 μs ±  15.744 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ██▇▅▅▅▃▄▄▃▃▂▂▂▂▂▂▁▁▁                                        ▂
  ▃▄███████████████████████▇▇▇▅▆▆▆▆▆▄▅▅▅▅▄▄▅▄▄▅▃▁▄▁▄▃▁▁▁▁▃▃▇▆▃▅ █
  254 μs        Histogram: log(frequency) by time        321 μs <

 Memory estimate: 8.00 KiB, allocs estimate: 1.

In [12]:
# triangular solve directly; save the cost of istril()
@benchmark LowerTriangular($A) \ $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   91.542 μs … 320.417 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):      93.458 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   101.238 μs ±  21.983 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ██▅▅▄▂▃▂▂▁▁                                      ▂▃ ▂▂▁▂▂     ▂
  ████████████▇▇▆▇▄▃▆▁▄▁▁▄▁▁▁▁▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██████████▇▇ █
  91.5 μs       Histogram: log(frequency) by time        167 μs <

 Memory estimate: 8.00 KiB, allocs estimate: 1.

## Block diagonal matrix

Block diagonal: Suppose $n = \sum_b n_b$. For linear equations, $(\sum_b n_b)^3$ (without using block diagonal structure) vs $\sum_b n_b^3$ (using block diagonal structure).  

Julia has a [`blockdiag`](https://docs.julialang.org/en/v1/stdlib/SparseArrays/#SparseArrays.blockdiag) function that generates a **sparse** matrix. **Anyone interested writing a `BlockDiagonal.jl` package?**

In [13]:
using SparseArrays

Random.seed!(257)

B  = 10 # number of blocks
ni = 100
A  = blockdiag([sprandn(ni, ni, 0.01) for b in 1:B]...)

1000×1000 SparseMatrixCSC{Float64, Int64} with 994 stored entries:
⎡⣨⣿⣶⡟⡅⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⢨⣾⣿⠼⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠈⠈⠀⠀⣿⣽⣃⡽⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⣻⣝⣶⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠈⠀⠉⠈⢶⣛⢮⣭⡁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⡺⡟⣮⡞⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣯⣾⢟⡾⠂⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣟⡩⣷⣏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠀⣿⣿⣿⣬⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣟⣿⢝⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣽⡶⣾⢷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢷⣻⣽⣗⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⡻⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣭⢿⡿⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣿⣻⣬⣟⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⡧⣱⢾⣎⡀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠨⣫⣗⣱⡿⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠐⣾⣞⡿⡑⣀⠀⣀⢀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢘⣿⣳⣟⣉⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⡟⣿⢝⠟⎦

In [14]:
using UnicodePlots
spy(A)

         ┌──────────────────────────────┐    
       1 │⢿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⠛⠛⢉⣴⡤⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠀⠀⠀⣯⣻⣷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⣿⢿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠛⠛⢫⣤⣤⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⢿⣟⠀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠋⠛⣦⣤⣤⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣻⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠛⠛⣤⢤⣤⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣟⣯⣟⡀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣯⣿⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠛⢫⣤⣤⡴│    
   1 000 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⡿│    
         └──────────────────────────────┘    
         ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀1 000⠀    
         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀994 ≠ 0⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    

## Kronecker product

Use
$$
\begin{eqnarray*}
    (\mathbf{A} \otimes \mathbf{B})^{-1} &=& \mathbf{A}^{-1} \otimes \mathbf{B}^{-1} \\
    (\mathbf{C}^T \otimes \mathbf{A}) \text{vec}(\mathbf{B}) &=& \text{vec}(\mathbf{A} \mathbf{B} \mathbf{C}) \\
    \text{det}(\mathbf{A} \otimes \mathbf{B}) &=& [\text{det}(\mathbf{A})]^p [\text{det}(\mathbf{B})]^m, \quad \mathbf{A} \in \mathbb{R}^{m \times m}, \mathbf{B} \in \mathbb{R}^{p \times p}
\end{eqnarray*}    
$$
to avoid forming and doing costly computation on the potentially huge Kronecker $\mathbf{A} \otimes \mathbf{B}$.

**Anyone interested writing a package?**

In [15]:
using MatrixDepot, LinearAlgebra

A = matrixdepot("lehmer", 50) # a pd matrix

┌ Info: verify download of index files...
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/MatrixDepot.jl:117
┌ Info: reading database
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/download.jl:24
┌ Info: adding metadata...
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/download.jl:68
┌ Info: adding svd data...
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/download.jl:70
┌ Info: writing database
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/download.jl:75
┌ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index
└ @ MatrixDepot /Users/huazhou/.julia/packages/MatrixDepot/5VE9J/src/MatrixDepot.jl:119


50×50 Matrix{Float64}:
 1.0        0.5        0.333333   0.25       …  0.0208333  0.0204082  0.02
 0.5        1.0        0.666667   0.5           0.0416667  0.0408163  0.04
 0.333333   0.666667   1.0        0.75          0.0625     0.0612245  0.06
 0.25       0.5        0.75       1.0           0.0833333  0.0816327  0.08
 0.2        0.4        0.6        0.8           0.104167   0.102041   0.1
 0.166667   0.333333   0.5        0.666667   …  0.125      0.122449   0.12
 0.142857   0.285714   0.428571   0.571429      0.145833   0.142857   0.14
 0.125      0.25       0.375      0.5           0.166667   0.163265   0.16
 0.111111   0.222222   0.333333   0.444444      0.1875     0.183673   0.18
 0.1        0.2        0.3        0.4           0.208333   0.204082   0.2
 ⋮                                           ⋱                        
 0.0238095  0.047619   0.0714286  0.0952381     0.875      0.857143   0.84
 0.0232558  0.0465116  0.0697674  0.0930233     0.895833   0.877551   0.86
 0.02272

In [16]:
B = matrixdepot("oscillate", 100) # pd matrix

100×100 Matrix{Float64}:
  0.707518      0.0578848    -0.00202508   …  -2.07618e-11   3.97326e-12
  0.0578848     0.433525      0.0956315        1.36333e-10  -2.60905e-11
 -0.00202508    0.0956315     0.807263        -1.45959e-10   2.79326e-11
  0.00464379   -0.0293052     0.0908958        8.28755e-10  -1.58602e-10
 -0.000796805   0.00596484   -0.00873108      -5.8795e-10    1.12518e-10
  0.000152015  -0.00249669    0.00519362   …   8.62146e-10  -1.64992e-10
  1.24633e-5    0.000187569  -0.00288517      -1.75966e-9    3.36752e-10
 -3.61382e-5   -3.52123e-5    0.000736403      1.38969e-9   -2.65949e-10
 -0.000252717   0.00129378   -0.000357227     -2.23571e-9    4.27855e-10
  2.10061e-5   -0.000141756   0.000188028      9.45806e-10  -1.81002e-10
  ⋮                                        ⋱                
  2.41765e-11  -1.58755e-10   1.69964e-10      0.000337715  -6.45464e-5
 -3.38657e-11   2.2238e-10   -2.38081e-10     -0.000474301   9.05451e-5
  4.06352e-12  -2.66832e-11   2.85671e-1

In [17]:
M = kron(A, B)
c = ones(size(M, 2)) # rhs
# Method 1: form Kronecker product and Cholesky solve
x1 = cholesky(Symmetric(M)) \ c;

In [18]:
# Method 2: use (A ⊗ B)^{-1} = A^{-1} ⊗ B^{-1}
m, p = size(A, 1), size(B, 1)
x2 = vec(transpose(cholesky(Symmetric(A)) \ 
    transpose(cholesky(Symmetric(B)) \ reshape(c, p, m))));

In [19]:
# relative error
norm(x1 - x2) / norm(x1)

5.823335144697673e-8

In [20]:
using BenchmarkTools

# Method 1: form Kronecker and Cholesky solve
@benchmark cholesky(Symmetric(kron($A, $B))) \ c

BenchmarkTools.Trial: 24 samples with 1 evaluation.
 Range (min … max):  206.014 ms … 247.471 ms  ┊ GC (min … max): 0.11% … 17.18%
 Time  (median):     211.256 ms               ┊ GC (median):    2.56%
 Time  (mean ± σ):   214.476 ms ±   9.686 ms  ┊ GC (mean ± σ):  3.88% ±  3.93%

        ▅█▂                                                      
  ▅▁▁▁▅█████▅▅▁▁▁▁▅▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▅ ▁
  206 ms           Histogram: frequency by time          247 ms <

 Memory estimate: 381.51 MiB, allocs estimate: 7.

In [21]:
# Method 2: use (A ⊗ B)^{-1} = A^{-1} ⊗ B^{-1}
@benchmark vec(transpose(cholesky(Symmetric($A)) \ 
    transpose(cholesky(Symmetric($B)) \ reshape($c, p, m))))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   77.709 μs …  10.213 ms  ┊ GC (min … max): 0.00% … 98.93%
 Time  (median):      88.042 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   102.481 μs ± 251.762 μs  ┊ GC (mean ± σ):  8.02% ±  3.40%

    ▁▅██▇▆▅▅▄▄▃▂▂▁   ▁ ▁▁▁  ▁▁                                  ▂
  █████████████████████████████▇▇▇▆▇▆▅▆▅▅▅▅▅▄▁▅▄▁▃▃▁▁▁▃▁▄▁▆▆▆▅▆ █
  77.7 μs       Histogram: log(frequency) by time        185 μs <

 Memory estimate: 176.42 KiB, allocs estimate: 15.

## Sparse matrix

Sparsity: sparse matrix decomposition or iterative method.  

* The easiest recognizable structure. Familiarize yourself with the sparse matrix computation tools in Julia, Matlab, R (`Matrix` package), MKL (sparse BLAS), ... as much as possible.

In [22]:
using MatrixDepot

Random.seed!(257)

# a 7701-by-7701 sparse pd matrix
A = matrixdepot("wathen", 50)
# random generated rhs
b = randn(size(A, 1))
Afull = Matrix(A)
count(!iszero, A) / length(A) # sparsity

0.001994776158751544

In [23]:
using UnicodePlots
spy(A)

         ┌──────────────────────────────┐    
       1 │⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ > 0
         │⠀⠙⢿⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│ < 0
         │⠀⠀⠀⠘⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠙⠻⣦⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣧⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣧⣄⠀⠀⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⡄⠀⠀⠀│    
         │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⢿⣷⣄⠀│    
   7 701 │⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠿⣧│    
         └──────────────────────────────┘    
         ⠀1⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀7 701⠀    
         ⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀118 301 ≠ 0⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀    

### Matrix-vector multiplication

In [24]:
# dense matrix-vector multiplication
@benchmark $Afull * $b

BenchmarkTools.Trial: 688 samples with 1 evaluation.
 Range (min … max):  6.703 ms …  10.519 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.226 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.262 ms ± 473.139 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

     ██▂                ▇▅                                     
  ▃▄▇███▆▃▃▃▃▃▃▃▅▄▄▃▃▃▃▄███▅▄▃▂▃▁▃▂▂▁▁▂▁▂▂▂▂▂▁▂▂▂▁▂▁▁▁▁▁▂▁▁▁▂ ▃
  6.7 ms          Histogram: frequency by time        8.94 ms <

 Memory estimate: 60.30 KiB, allocs estimate: 2.

In [25]:
# sparse matrix-vector multiplication
@benchmark $A * $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  62.083 μs …  9.586 ms  ┊ GC (min … max): 0.00% … 98.91%
 Time  (median):     66.708 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   69.822 μs ± 95.301 μs  ┊ GC (mean ± σ):  1.36% ±  0.99%

    ▁   ▂▆██▇▆▄▄▄▃▄▄▄▄▃▂▂▁▁▂▂▂▂▂▃▂▁▁ ▁  ▁                     ▂
  ▄▆██▇▇████████████████████████████████████▇█▇▇▇██▇▆▅▅▆▆▆▆▆▅ █
  62.1 μs      Histogram: log(frequency) by time      88.3 μs <

 Memory estimate: 60.30 KiB, allocs estimate: 2.

### Solve linear equation

In [26]:
# solve via dense Cholesky
xchol = cholesky(Symmetric(Afull)) \ b
@benchmark cholesky($(Symmetric(Afull))) \ $b

BenchmarkTools.Trial: 8 samples with 1 evaluation.
 Range (min … max):  634.202 ms … 706.123 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     653.264 ms               ┊ GC (median):    0.50%
 Time  (mean ± σ):   654.889 ms ±  22.750 ms  ┊ GC (mean ± σ):  0.85% ± 0.96%

  ▁ ▁ ▁           █  ▁▁                                       ▁  
  █▁█▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  634 ms           Histogram: frequency by time          706 ms <

 Memory estimate: 452.52 MiB, allocs estimate: 4.

In [27]:
# solve via sparse Cholesky
xcholsp = cholesky(Symmetric(A)) \ b
@show norm(xchol - xcholsp)
@benchmark cholesky($(Symmetric(A))) \ $b

norm(xchol - xcholsp) = 4.167541001014585e-15


BenchmarkTools.Trial: 776 samples with 1 evaluation.
 Range (min … max):  5.971 ms … 13.758 ms  ┊ GC (min … max): 0.00% … 1.52%
 Time  (median):     6.234 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.443 ms ±  1.012 ms  ┊ GC (mean ± σ):  0.08% ± 0.27%

  ▇█▇▆▅▃▁                                                     
  ███████▆▇▆▄▄▆▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▁▁▁▄▄▄▁▁▁▁▆▁▁▄▅▁▄▁▁▁▁▄▁▄▁▁▄▅ ▇
  5.97 ms      Histogram: log(frequency) by time     12.8 ms <

 Memory estimate: 12.43 MiB, allocs estimate: 64.

In [28]:
# sparse solve via conjugate gradient
using IterativeSolvers

xcg = cg(A, b)
@show norm(xcg - xchol)
@benchmark cg($A, $b)

norm(xcg - xchol) = 2.5745373086732674e-7


BenchmarkTools.Trial: 306 samples with 1 evaluation.
 Range (min … max):  16.025 ms …  20.564 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.225 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.375 ms ± 419.693 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▂▃▂▇█ ▃▂                                                    
  ▆▄█████▆██▆▄▄▄▄▄▃▄▅▃▃▃▆▃▄▂▅▂▆▃▅▃▄▂▅▄▃▃▃▂▂▂▃▁▂▃▁▂▁▁▂▂▁▁▁▁▁▁▁▃ ▃
  16 ms           Histogram: frequency by time         17.4 ms <

 Memory estimate: 241.89 KiB, allocs estimate: 16.

In [29]:
# sparse solve via preconditioned conjugate gradient
using Preconditioners

xpcg = cg(A, b, Pl = DiagonalPreconditioner(A))
@show norm(xpcg - xchol)
@benchmark cg($A, $b, Pl = $(DiagonalPreconditioner(A)))

norm(xpcg - xchol) = 3.572940522731538e-8


BenchmarkTools.Trial: 1454 samples with 1 evaluation.
 Range (min … max):  3.306 ms …   8.778 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.395 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.438 ms ± 257.696 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▅█▇▄▄▂▁                                                    
  ▂▅███████▇▇▇▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▂▁▁▂▂▁▂▁▁▁▂▂▁▁▁▂▂▂ ▃
  3.31 ms         Histogram: frequency by time        4.09 ms <

 Memory estimate: 241.89 KiB, allocs estimate: 16.

## Easy plus low rank

Easy plus low rank: $\mathbf{U} \in \mathbb{R}^{n \times r}$, $\mathbf{V} \in \mathbb{R}^{r \times n}$, $r \ll n$. Woodbury formula
\begin{eqnarray*}
	(\mathbf{A} + \mathbf{U} \mathbf{V}^T)^{-1} &=& \mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{U} (\mathbf{I}_r + \mathbf{V}^T \mathbf{A}^{-1} \mathbf{U})^{-1} \mathbf{V}^T \mathbf{A}^{-1} \\
    \text{det}(\mathbf{A} + \mathbf{U} \mathbf{V}^T) &=& \text{det}(\mathbf{A}) \text{det}(\mathbf{I}_r + \mathbf{V} \mathbf{A}^{-1} \mathbf{U}^T).
\end{eqnarray*}

* Keep HW3 (multivariate density) and HW4 (PageRank) problems in mind.  

* [`WoodburyMatrices.jl`](https://github.com/timholy/WoodburyMatrices.jl) package can be useful.

In [30]:
using BenchmarkTools, Random, WoodburyMatrices

Random.seed!(257)
n = 1000
r = 5

A = Diagonal(rand(n))
B = randn(n, r)
D = Diagonal(rand(r))
b = randn(n)
# Woodbury structure: W = A + B * D * B'
W = SymWoodbury(A, B, D)
Wfull = Matrix(W) # stored as a Matrix{Float64}

1000×1000 Matrix{Float64}:
  1.59654    0.456106    1.52748   …  -0.725063   1.23432   -0.467818
  0.456106   3.23966     0.960678      1.22398   -0.256551  -0.191276
  1.52748    0.960678    3.39075       0.432445   1.34648   -1.06645
 -0.486356  -0.0292724  -1.36154      -0.998257  -0.378862   0.63341
 -1.12244    0.777027   -1.45956       1.42378   -1.84637    0.509516
  0.519072   1.93077     1.4578    …   1.88547   -0.250131  -0.539974
 -1.31967   -0.941968   -1.55701       1.24036   -1.73289    0.33383
 -0.365699   1.79832    -0.749824      1.03221   -0.893575   0.393119
 -0.206288  -1.30162    -0.764563     -1.75899    0.249943   0.334384
  0.598281   1.96579     2.04488       2.91997   -0.423147  -0.897774
  ⋮                                ⋱                        
  1.32891    1.73318     1.88389      -0.454571   1.60264   -0.38926
 -0.855297   2.71166    -0.811348      2.49806   -2.44808    0.339342
  0.101575   2.2306     -0.13939       0.201285  -0.590535   0.288461
 -1.03

In [31]:
# compares storage
Base.summarysize(W), Base.summarysize(Wfull)

(48480, 8000040)

### Solve linear equation

In [32]:
# solve via Cholesky
@benchmark cholesky($(Symmetric(Wfull))) \ $b

BenchmarkTools.Trial: 1312 samples with 1 evaluation.
 Range (min … max):  2.904 ms … 30.826 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.336 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.810 ms ±  1.865 ms  ┊ GC (mean ± σ):  4.05% ± 9.53%

    █▆▃▁                                                      
  ▆▃█████▇█▇▆▅▄▃▄▅▇▇▇▄▅▅▅▅▅▄▅▃▁▄▄▃▁▁▁▄▁▁▃▁▃▁▁▃▄▄▃▆▆▃▃▄▃▁▃▄▁▃ █
  2.9 ms       Histogram: log(frequency) by time     11.5 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

In [33]:
# solve using Woodbury formula
@benchmark $W \ reshape($b, n, 1) # hack; need to file an issue 

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.132 μs …  3.201 ms  ┊ GC (min … max):  0.00% … 99.68%
 Time  (median):     7.097 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.741 μs ± 64.346 μs  ┊ GC (mean ± σ):  16.42% ±  2.23%

                        █▃                                    
  ▂▂▂▂▂▂▁▁▁▁▂▂▂▂▂▂▂▂▃▃▅███▆▆▆▅▆▆▅▄▅▄▃▃▃▃▃▃▃▃▃▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂ ▃
  5.13 μs        Histogram: frequency by time        9.58 μs <

 Memory estimate: 32.28 KiB, allocs estimate: 8.

### Matrix-vector multiplication

In [34]:
# multiplication without using Woodbury structure
@benchmark $Wfull * $b

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   86.292 μs …  6.432 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     100.416 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   108.562 μs ± 73.301 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▁▅█▄▁▁▃▃▃▃▃                                          ▁     ▁
  ▄▇████████████▅▃▄▁▃▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▃▃█▇ █
  86.3 μs       Histogram: log(frequency) by time       273 μs <

 Memory estimate: 8.00 KiB, allocs estimate: 1.

In [35]:
# multiplication using Woodbury structure
@benchmark $W * $b

BenchmarkTools.Trial: 10000 samples with 8 evaluations.
 Range (min … max):  2.516 μs …  2.242 ms  ┊ GC (min … max):  0.00% … 99.70%
 Time  (median):     4.458 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.924 μs ± 50.189 μs  ┊ GC (mean ± σ):  20.65% ±  2.44%

                           ▄█▂▂▂                              
  ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▃▃▄█████▇▆▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁ ▂
  2.52 μs        Histogram: frequency by time        6.36 μs <

 Memory estimate: 32.19 KiB, allocs estimate: 6.

### Determinant

In [36]:
# determinant without using Woodbury structure
@benchmark det($Wfull)

BenchmarkTools.Trial: 1331 samples with 1 evaluation.
 Range (min … max):  3.175 ms … 16.567 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     3.566 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.756 ms ±  1.150 ms  ┊ GC (mean ± σ):  3.85% ± 9.31%

    █▇                                                        
  ▆▃███▅▄▄▃▁▁▁▃▁▁▁▁▁▃▁▁▁▁▁▁▁▁▃▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▃▅▆▆ █
  3.18 ms      Histogram: log(frequency) by time     10.9 ms <

 Memory estimate: 7.64 MiB, allocs estimate: 3.

In [37]:
# determinant using Woodbury structure
@benchmark det($W)

BenchmarkTools.Trial: 10000 samples with 195 evaluations.
 Range (min … max):  458.759 ns … 103.921 μs  ┊ GC (min … max): 0.00% … 99.36%
 Time  (median):     487.605 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   506.929 ns ±   1.035 μs  ┊ GC (mean ± σ):  2.04% ±  0.99%

           ▁▆█▆▆▂                                                
  ▁▃▃▂▁▁▁▃▇███████▆▅▄▄▄▄▃▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  459 ns           Histogram: frequency by time          585 ns <

 Memory estimate: 352 bytes, allocs estimate: 2.

## Easy plus border

Easy plus border: For $\mathbf{A}$ pd and $\mathbf{V}$ full row rank,
$$
	\begin{pmatrix}
	\mathbf{A} & \mathbf{V}^T \\
	\mathbf{V} & \mathbf{0}
	\end{pmatrix}^{-1} = \begin{pmatrix}
	\mathbf{A}^{-1} - \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & \mathbf{A}^{-1} \mathbf{V}^T (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \\
	(\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1} \mathbf{V} \mathbf{A}^{-1} & - (\mathbf{V} \mathbf{A}^{-1} \mathbf{V}^T)^{-1}
	\end{pmatrix}.
$$
**Anyone interested writing a package?**

## Orthogonal matrix

Orthogonal $\mathbf{A}$: $n^2$ flops **at most**. Why? Permutation matrix, Householder matrix, Jacobi matrix, ... take less.

## Toeplitz matrix

Toeplitz systems (constant diagonals):
$$
	\mathbf{T} = \begin{pmatrix}
	r_0 & r_1 & r_2 & r_3 \\
	r_{-1} & r_0 & r_1 & r_2 \\
	r_{-2} & r_{-1} & r_0 & r_1 \\
	r_{-3} & r_{-2} & r_{-1} & r_0
	\end{pmatrix}.
$$
$\mathbf{T} \mathbf{x} = \mathbf{b}$, where $\mathbf{T}$ is pd and Toeplitz, can be solved in $O(n^2)$ flops. Durbin algorithm (Yule-Walker equation), Levinson algorithm (general $\mathbf{b}$), Trench algorithm (inverse). These matrices occur in auto-regressive models and econometrics.

* [`ToeplitzMatrices.jl`](https://github.com/JuliaMatrices/ToeplitzMatrices.jl) package can be useful.

## Circulant matrix

Circulant systems: Toeplitz matrix with wraparound
$$
	C(\mathbf{z}) = \begin{pmatrix}
	z_0 & z_4 & z_3 & z_2 & z_1 \\
	z_1 & z_0 & z_4 & z_3 & z_2 \\
	z_2 & z_1 & z_0 & z_4 & z_3 \\
	z_3 & z_2 & z_1 & z_0 & z_4 \\
	z_4 & z_3 & z_2 & z_1 & z_0
	\end{pmatrix},
$$
FFT type algorithms: DCT (discrete cosine transform) and DST (discrete sine transform).

## Vandermonde matrix

Vandermonde matrix: such as in interpolation and approximation problems
$$
	\mathbf{V}(x_0,\ldots,x_n) = \begin{pmatrix}
	1 & 1 & \cdots & 1 \\
	x_0 & x_1 & \cdots & x_n \\
	\vdots & \vdots & & \vdots \\
	x_0^n & x_1^n & \cdots & x_n^n
	\end{pmatrix}.
$$
$\mathbf{V} \mathbf{x} = \mathbf{b}$ or $\mathbf{V}^T \mathbf{x} = \mathbf{b}$ can be solved in $O(n^2)$ flops.

## Cauchy-like matrix

Cauchy-like matrices:
$$
	\Omega \mathbf{A} - \mathbf{A} \Lambda = \mathbf{R} \mathbf{S}^T,
$$
where $\Omega = \text{diag}(\omega_1,\ldots,\omega_n)$ and $\Lambda = \text{diag}(\lambda_1,\ldots, \lambda_n)$. $O(n)$ flops for LU and QR.

## Structured-rank matrix

Structured-rank problems: semiseparable matrices (LU and QR takes $O(n)$ flops), quasiseparable matrices, ...

# LinearSolve.jl package

[LinearSolve.jl](https://github.com/SciML/LinearSolve.jl) is meta-package in Julia that defines a unified interface for solving linear equations and makes it easy switching linear solvers while maintaining the maximum efficiency.

TODO: examples.